In [2]:
import json
import os
import cv2
import glob
from tqdm.auto import tqdm
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from tqdm import tqdm
import shutil

In [ ]:
test_video_path = '/workspace/data/Test/*'
label_dir_path = '/workspace/Final_Submission/post_sota_ensemble/'
save_path = '/workspace/Final_Submission/videos_sota/'
font =  cv2.FONT_HERSHEY_PLAIN
video_paths = glob.glob(test_video_path)
for video_path in video_paths:
    video_name = os.path.basename(video_path)
    img_paths = glob.glob(video_path + '/img1/*.jpg')
    label_path = label_dir_path + video_name + '.txt'
    frames = []
    f = open(label_path, 'r')
    annos = []
    for line in f.readlines():
        annos.append(line.strip())
    
    for idx, path in tqdm(enumerate(img_paths)):
        bboxes = list(filter(lambda x : x.split(",")[0] == str(idx+1),annos))
        image = cv2.imread(path)
        for bbox  in bboxes:
            l = list(map(float,bbox.split(',')))
            l = list(map(int,l))
            [frame, id, bb_left, bb_top, width, height, confidence, _, _, _] = l
            image = cv2.rectangle(image,(bb_left,bb_top),(bb_left + width,bb_top + height),(255,255,255),3)
            image = cv2.putText(image, str(id), (bb_left,bb_top), font, 2, (255, 0, 0), 2, cv2.LINE_AA)
        frames.append(image)
    height, width, layers = image.shape
    size = (width,height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(save_path+video_name+'_video.mp4',fourcc, 20, size)
    for i in range(len(frames)):
        out.write(frames[i])
    out.release()